In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

import h5py
from datetime import datetime
import os

# Concepts
Since I am in a spherically symmetric spacetime, I can fix the angles to arbitrary values and forget them. So I only need to keep track of the radius. The 3D problem is effectively reduced to a 1D one!!! Radial differenciation is then reduced to simply the derivative along the array.

In [14]:
class Fields():
    
    def __init__(self, R = 10, N = 110):
        # Domain is a sphere of radius R
        self.Rmax = R
        self.N    = N
        
        self.dR  = R / N
    
        self.r = np.array([(j - 0.5)*self.dR for j in range(1,N+1)], dtype = np.float64)

        self.A   = np.zeros(N)
        self.B   = np.zeros(N)
        self.DA  = np.zeros(N)
        self.DB  = np.zeros(N)
        self.KA  = np.zeros(N)
        self.KB  = np.zeros(N)
        self.al  = np.zeros(N)
        
        self.psi = 1 + 1 / 4 / self.r  #r_s = 1
        
    def IC_1plusLogSlicing(self):
        """
        1+Log Slicing gauge condition.
                A = B = psi
                DA = DB = d/dr ln(x), x = A, B
                KA = KB = 0
                alpha   = 1
        """
        self.A  = np.copy(self.psi)
        self.B  = np.copy(self.psi)
        for i in range(N):
            self.DA[i] = der_r(np.log(self.A), i, self.dR)
            self.DB[i] = der_r(np.log(self.B), i, self.dR)
        self.al = np.ones(self.N)
        
    def a_symmetryCond(self):
        self.A  [0] = self.A  [1]
        self.B  [0] = self.B  [1]
        self.DA [0] = -self.DA[1]
        self.DB [0] = -self.DB[1]
        self.KA [0] = self.KA [1]
        self.KB [0] = self.KB [1]
        self.al [0] = self.al [1]
        
    def BC_Dirichlet(self):
        self.A[-1] = 1
        self.B[-1] = 1
        
        self.DA[-1] = 0
        self.DB[-1] = 0
        
    def BC_fstInterp(self):
        self.A[-1] = self.A[-3] + (self.A[-2] - self.A[-3]) * (self.r[-1] - self.r[-3]) / (self.r[-2] - self.r[-3])
        self.B[-1] = self.B[-3] + (self.B[-2] - self.B[-3]) * (self.r[-1] - self.r[-3]) / (self.r[-2] - self.r[-3])
        
        self.KA[-1] = self.KA[-3] + (self.KA[-2] - self.KA[-3]) * (self.r[-1] - self.r[-3]) / (self.r[-2] - self.r[-3])
        self.KB[-1] = self.KB[-3] + (self.KB[-2] - self.KB[-3]) * (self.r[-1] - self.r[-3]) / (self.r[-2] - self.r[-3])
        
        self.DA[-1] = self.DA[-3] + (self.DA[-2] - self.DA[-3]) * (self.r[-1] - self.r[-3]) / (self.r[-2] - self.r[-3])
        self.DB[-1] = self.DB[-3] + (self.DB[-2] - self.DB[-3]) * (self.r[-1] - self.r[-3]) / (self.r[-2] - self.r[-3])
        
        self.al[-1] = self.al[-3] + (self.al[-2] - self.al[-3]) * (self.r[-1] - self.r[-3]) / (self.r[-2] - self.r[-3])

In [3]:
def der_r(f, i, dR):
    if i == 0:
        # Forward
        return 0.5 * (- 3 * f[0] + 4 * f[1] - f[2]) / dR
    elif i == len(f)-1:
        # Backward
        return 0.5 * (3 * f[-1] - 4 * f[-2] + f[-3]) / dR
    else:
        # Central
        return 0.5 * (f[i+1] - f[i-1]) / dR

def sec_der_r(f, i, dR):
    if i == 0:
        # Forward
        return (2 * f[0] - 5 * f[1] + 4 * f[2] - f[3]) / dR**3
    elif i == len(f)-1:
        # Backward
        return (2 * f[-1] -5 * f[-2] + 4 * f[-3] - f[-4]) / dR**3
    else:
        # Central
        return (f[i+1] - 2 * f[i] + f[i-1]) / dR**2

In [56]:
def ev_A(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i] * field.A[i] * field.KA[i]

def ev_B(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i] * field.B[i] * field.KB[i]

def ev_DA(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KA[i] * der_r(np.log(field.al), i, field.dR)
    p2 = der_r(field.KA, i, field.dR)
    return - 2 * field.al[i] * (p1 + p2)

def ev_DB(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KB[i] * der_r(np.log(field.al), i, field.dR)
    p2 = der_r(field.KB, i, field.dR)
    return - 2 * field.al[i] * (p1 + p2)


def ev_KA(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    r = field.r[i]
    Dal = [der_r(np.log(field.al), j, field.dR) for j in range(field.N)]
    
    p1 = der_r(Dal + field.DB, i, field.dR)
    p2 = Dal[i]**2 + 0.5 * (- Dal[i] * field.DA[i] + field.DB[i]**2 - field.DA[i] * field.DB[i])
    p3 = - field.A[i] * field.KA[i] * (field.KA[i] + 2*field.KB[i])
    p4 = - (field.DA[i] - 2 * field.DB[i]) / r
    return - field.al[i] * (p1 + p2 + p3 + p4) / field.A[i]
    
def ev_KB(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    r = field.r[i]
    
    p1 = der_r(field.DB, i, field.dR)
    p2 = der_r(np.log(field.al), i, field.dR) * field.DB[i] + field.DB[i]**2 - 0.5 * field.DA[i] * field.DB[i]
    p3 = - (field.DA[i] - 2 * der_r(np.log(field.al), i, field.dR) - 4 * field.DB[i]) / r
    p4 = - 2 * (field.A[i] - field.B[i]) / (field.B[i] * r**2)
    p6 = field.al[i] * field.KB[i] * (field.KA[i] + 2*field.KB[i])
    return - 0.5 * field.al[i] * (p1 + p2 + p3 + p4) / field.A[i] + p6

def ev_al(field, i):
    return -2 * field.al[i] * (field.KA[i] + field.KB[i]*2)

In [57]:
def func(x, field):
    dB = np.zeros(field.N)
    for i in range(field.N):
        dB[i] = der_r(field.B, i, field.dR)
    tab = (2 / x + dB / field.B) / np.sqrt(field.A) - 2 * field.KB
    
    return f

def comp_appHorizon(field):
    root = fsolve(func, 1., args=(field,), full_output=True)
    return root

In [62]:
SAVE = True
dt = 0.0001
t = 0
tmax = 1000
N = 20
R = 5

if SAVE:
    name = f'{datetime.now().strftime("%Y_%m_%d_%H_%M_%S")}_1pL_res.h5'
    h5f = h5py.File(name, 'w')
    sss = f'Each dataset contains the following variables:\n \
    \tA\n\
    \tB\n\
    \tD_A\n\
    \tD_B\n\
    \tK_A\n\
    \tK_B\n\
    \tD_alpha\n\
    \talpha\n\
    \nThe name of the dataset is the index of the time step (e.g. "i" is the first computation, at t = i*dt)\n\
    Parameters:\n\
    \tN = {N} (number of points in the radial direction)\n\
    \tdt = {dt} (timestep used)\n\
    \tt_max = {tmax} (number of time iterations)'
    h5f.attrs['Info']  = sss

fields = Fields(R = R, N = N)
fields.IC_1plusLogSlicing()

results = np.zeros((7, N))

for j in range(tmax):
    t += dt
    
    fields.a_symmetryCond()
    fields.BC_fstInterp()
    
    results[0,0]  = fields.A  [0]
    results[1,0]  = fields.B  [0]
    results[2,0]  = fields.DA [0]
    results[3,0]  = fields.DB [0]
    results[4,0]  = fields.KA [0]
    results[5,0]  = fields.KB [0]
    results[6,0]  = fields.al [0]
    
    results[0,-1] = fields.A  [-1]
    results[1,-1] = fields.B  [-1]
    results[2,-1] = fields.DA [-1]
    results[3,-1] = fields.DB [-1]
    results[4,-1] = fields.KA [-1]
    results[5,-1] = fields.KB [-1]
    results[6,-1] = fields.al [-1]
    
    for i in range(1,N-1):

        results[0,i] = fields.A [i] + dt * ev_A (fields, i) #  A_new  
        results[1,i] = fields.B [i] + dt * ev_B (fields, i) #  B_new 
        results[2,i] = fields.DA[i] + dt * ev_DA(fields, i) #  DA_new
        results[3,i] = fields.DB[i] + dt * ev_DB(fields, i) #  DB_new 
        results[4,i] = fields.KA[i] + dt * ev_KA(fields, i) #  KA_new 
        results[5,i] = fields.KB[i] + dt * ev_KB(fields, i) #  KB_new 
        results[6,i] = fields.al[i] + dt * ev_al(fields, i) #  al_new

    if SAVE:
        h5f.create_dataset(f'{j}', data=results, compression = 9)
    
    fields.A  = np.copy(results[0,:])
    fields.B  = np.copy(results[1,:])
    fields.DA = np.copy(results[2,:])
    fields.DB = np.copy(results[3,:])
    fields.KA = np.copy(results[4,:])
    fields.KB = np.copy(results[5,:])
    fields.al = np.copy(results[6,:])

    #rh = comp_appHorizon(fields)
    #print(rh)
    
if SAVE:    
    h5f.close()
    os.system(f'python3 Plots_1plusLog.py {name} {N} {tmax} {fields.dR}')

MovieWriter ffmpeg unavailable; using Pillow instead.


Figure(640x480)


In [61]:
dB = np.zeros(fields.N)
for i in range(fields.N):
    dB[i] = der_r(fields.B, i, fields.dR)
    
tab = (2 / fields.r + dB / fields.B) / np.sqrt(fields.A) - 2 * fields.KB
for i in range(len(tab)-1):
    if np.sign(tab[i]) != np.sign(tab[i+1]):
        print(f'Change at idx {i}-{i+1}: {tab[i]} {tab[i+1]}')

Change at idx 2-3: 29.640400276309478 -23.136323867348835
Change at idx 3-4: -23.136323867348835 7.067369982320762


In [ ]:
f = [field.A[0]...field.A[N]field.B[0]...field.al[N]]